In [13]:
%pip install nltk gensim transformers
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
import torch

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [16]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize stop words, lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# File paths
filepaths = [
    'organized_blow_molding_basics.csv',
    'Dupont_Table7.csv',
    'organized_pet_bottle_faults.csv',
    'organized_stressesinpebottles.csv'
]

# Function to load and concatenate text data from each CSV file
def load_text_from_csv(filepaths, text_column):
    text_data = []
    for filepath in filepaths:
        df = pd.read_csv(filepath)
        text_data.extend(df[text_column].dropna().tolist())
    return text_data

# Specify the column name containing text in your CSVs
text_column = 'text'  # Replace 'text' with the actual name of your text column

# Load and concatenate text data from CSV files
texts = load_text_from_csv(filepaths, text_column)

# Preprocessing function
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stop words and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
    return tokens

# Preprocess all text data
processed_texts = [preprocess_text(text) for text in texts]

# Word2Vec embeddings
word2vec_model = Word2Vec(sentences=processed_texts, vector_size=100, window=5, min_count=1, workers=4)
print("Word2Vec Embedding for 'example_word':", word2vec_model.wv['example_word'])  # replace 'example_word' with any word in your text

# BERT embeddings using Hugging Face's transformers
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = bert_model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Sentence-level embedding
    return embeddings

# Generate BERT embeddings for each processed text file
bert_embeddings = [get_bert_embeddings(" ".join(tokens)) for tokens in processed_texts]

# Display an example
print("BERT Embedding for first document:", bert_embeddings[0].detach().numpy())


[nltk_data] Downloading package punkt to /Users/eunsoe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/eunsoe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/eunsoe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


KeyError: 'text'